# EURUSD prediction

In [20]:
from database_operations.DatabaseManager import DatabaseManager
from scripts.HistoricalDataCollector import HistoricalDataCollector
from xapi import PeriodCode
from my_secrets import config

import pandas as pd

### Connection to database and define symbol

In [8]:
SYMBOL = "EURUSD"
database_conn = DatabaseManager(config.DB_FILE)

Disconnected from db due to deletion of the DatabaseConnector object.


In [9]:
SYMBOL_ID  = await database_conn.get_specify_data("stock",["id"], f"symbol = ?",(SYMBOL,))
SYMBOL_ID = SYMBOL_ID[0][0]

Connected to db.
Set row factory.


## Download newest EURUSD data and save in db

In [10]:
hist_data_collector = HistoricalDataCollector(
    symbol= SYMBOL,
    start='2000-01-01',
    end='2023-10-10',
    period=PeriodCode.PERIOD_D1,
    credentials_file=config.CREDENTIALS_PATH
)
await hist_data_collector.connect_to_xapi()
await database_conn.populate_prices("stock_price_1d", symbol_id, SYMBOL, hist_data_collector)

Connected to API.
Last date in the database
2023-10-10 00:00:00
Today's date
2023-10-10 00:00:00
Newest data for symbol: EURUSD already exist in the database.
Disconnected from db.


## Download data from db

In [31]:
COLUMNS = ['date', 'open', 'close', 'high', 'low', 'volume']
columns_with_prefix = [f'sp.{column}' for column in COLUMNS]
columns_list = ', '.join(columns_with_prefix)

In [34]:
QUERY = f"""
        SELECT {columns_list}
        FROM stock s
        JOIN stock_price_1d sp ON s.id = sp.stock_id
        WHERE s.symbol = ?
        """

In [38]:
data_list = await database_conn.execute_custom_query(QUERY, (SYMBOL,))

## Create dataframe

In [41]:
data_df = pd.DataFrame(data_list, columns=COLUMNS)

In [43]:
print(data_df)

                     date     open    close     high      low  volume
0     2010-11-03 00:00:00  1.40308  1.40309  1.41816  1.25873       0
1     2010-11-16 00:00:00  1.35782  1.40309  1.41816  1.25873       0
2     2010-11-17 00:00:00  1.34837  1.35290  1.35649  1.34601       0
3     2010-12-16 00:00:00  1.32225  1.35290  1.35649  1.31804       0
4     2010-12-21 00:00:00  1.31183  1.31193  1.33583  1.30728       0
...                   ...      ...      ...      ...      ...     ...
3435  2023-10-05 00:00:00  1.05060  1.05480  1.05512  1.04994  957605
3436  2023-10-06 00:00:00  1.05478  1.05891  1.05995  1.04821  990975
3437  2023-10-08 00:00:00  1.05488  1.05538  1.05575  1.05352    2649
3438  2023-10-09 00:00:00  1.05550  1.05653  1.05738  1.05189  915355
3439  2023-10-10 00:00:00  1.05649  1.06092  1.06105  1.05538  448633

[3440 rows x 6 columns]
